In [1]:
import cali_interactive_explorer as cali
import pandas as pd

In [25]:
cali_frame = cali.CaliperFrame("SELECT *","/Users/poliakoff1/allocations/allocation_cal*.cali")
df = cali_frame.results

In [70]:
functions_with_cycles = {}
def alloc_cycle_checker(blackboard_name,current_blackboard,processing_row):
    # blackboard_name : We're grouping on annotations. This is the current values of that annotation set
    # current_blackboard : A scratchpad in which to maintain state in that set of values
    # processing_row : The current event in the trace we're examining
    func_val = processing_row["Returned_Field"]
    # If we're seeing an allocation event
    if not pandas.isnull(func_val):
        if "allocations" not in current_blackboard:
            current_blackboard["allocations"] = set([])
            current_blackboard["alloc_map"] = {}
        # Say it was allocated here
        current_blackboard["allocations"].add(func_val)
        lineinfo=zip(processing_row["source.file#callpath.address"].split("//"),processing_row["source.line#callpath.address"].split("/"),processing_row["source.function#callpath.address"].split("/"))
        current_blackboard["alloc_map"][func_val] = [(filename,lineno,func) for filename, lineno,func in lineinfo if "usr/" not in filename and "UNKNOWN" not in filename and "caliper/src" not in filename and "4cali" not in func and "cali::" not in func]
    
    
    # If we're seeing a free event
    func_val = processing_row["Field To Free"]
    if not pandas.isnull(func_val):
        if "allocations" not in current_blackboard:
            current_blackboard["allocations"] = set([])
        # If it was allocated in this blackboard state
        if func_val in current_blackboard["allocations"]:
            function_name = blackboard_name["function"]
            if function_name not in functions_with_cycles:
                functions_with_cycles[function_name] = { "pointers" : {},
                                                         "total" : 0}
                
            function_record = functions_with_cycles[function_name]
            function_record["total"] = function_record["total"] + 1
            if func_val not in function_record["pointers"]:
                function_record["pointers"][func_val] = 0
            function_record["pointers"][func_val] += 1
            raw_free_lineinfo=zip(processing_row["source.file#callpath.address"].split("//"),processing_row["source.line#callpath.address"].split("/"),processing_row["source.function#callpath.address"].split("/"))
            free_lineinfo=[(filename,lineno,func) for filename, lineno,func in raw_free_lineinfo if "usr/" not in filename and "UNKNOWN" not in filename and "caliper/src" not in filename and "4cali" not in func and "cali::" not in func]
            alloc_lineinfo = current_blackboard["alloc_map"][func_val]
            print "==========================================================================="
            for index,(filename,lineno,function) in enumerate(alloc_lineinfo):
             print "%s%s:%s" % ("  "*(index),filename,lineno)
            print ("=")
            print ("=")
            print ("V")
            for index,(filename,lineno,function) in enumerate(free_lineinfo):
             print "%s%s:%s" % ("  "*(index),filename,lineno)
            print "==========================================================================="
cali.process_trace(df,["function"],alloc_cycle_checker)

#print functions_with_cycles  

g/g0/dzpolia/src/caliper/examples/apps/cali-functional-memory.cpp:65
  g/g0/dzpolia/src/caliper/examples/apps/cali-functional-memory.cpp:52
=
=
V
g/g0/dzpolia/src/caliper/examples/apps/cali-functional-memory.cpp:65
  g/g0/dzpolia/src/caliper/examples/apps/cali-functional-memory.cpp:56
g/g0/dzpolia/src/caliper/examples/apps/cali-functional-memory.cpp:65
  g/g0/dzpolia/src/caliper/examples/apps/cali-functional-memory.cpp:52
=
=
V
g/g0/dzpolia/src/caliper/examples/apps/cali-functional-memory.cpp:65
  g/g0/dzpolia/src/caliper/examples/apps/cali-functional-memory.cpp:56
g/g0/dzpolia/src/caliper/examples/apps/cali-functional-memory.cpp:65
  g/g0/dzpolia/src/caliper/examples/apps/cali-functional-memory.cpp:52
=
=
V
g/g0/dzpolia/src/caliper/examples/apps/cali-functional-memory.cpp:65
  g/g0/dzpolia/src/caliper/examples/apps/cali-functional-memory.cpp:56
g/g0/dzpolia/src/caliper/examples/apps/cali-functional-memory.cpp:65
  g/g0/dzpolia/src/caliper/examples/apps/cali-functional-memory.cpp:52
=


Index([u'Field To Free', u'Field_Allocation_Size', u'Returned_Field',
       u'cali.caliper.version', u'cali.event.begin', u'cali.event.end',
       u'cali.event.set', u'callpath.address', u'event.begin#function',
       u'event.begin#wrapped_function', u'event.end#Field To Free',
       u'event.end#Field_Allocation_Size', u'event.end#Returned_Field',
       u'event.end#function', u'event.end#wrapped_function',
       u'event.set#Field To Free', u'event.set#Field_Allocation_Size',
       u'event.set#Returned_Field', u'function',
       u'source.file#callpath.address', u'source.function#callpath.address',
       u'source.line#callpath.address', u'time.inclusive.duration',
       u'wrapped_function'],
      dtype='object')